# Data wrangling

# 1. Import  packages and dataset

In [64]:
import pandas as pd
import numpy as np
import math
from collections import Counter
import re
import json
import requests
from bs4 import BeautifulSoup
import datetime

In [65]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [3]:
df = pd.read_csv('movie_metadata.csv')


# 2. Checking data's shape and overall information

In [4]:
df.shape

(5043, 28)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
color                        5024 non-null object
director_name                4939 non-null object
num_critic_for_reviews       4993 non-null float64
duration                     5028 non-null float64
director_facebook_likes      4939 non-null float64
actor_3_facebook_likes       5020 non-null float64
actor_2_name                 5030 non-null object
actor_1_facebook_likes       5036 non-null float64
gross                        4159 non-null float64
genres                       5043 non-null object
actor_1_name                 5036 non-null object
movie_title                  5043 non-null object
num_voted_users              5043 non-null int64
cast_total_facebook_likes    5043 non-null int64
actor_3_name                 5020 non-null object
facenumber_in_poster         5030 non-null float64
plot_keywords                4890 non-null object
movie_imdb_link              5043 non-

# 3. Check missing values in columns or rows

In [6]:
# iterate each col; check how many null in each variable
for col in df:
    null = df[col].isnull().sum()
    if null > 0:
        print col, null, '{0:.2%}'.format(float(null)/float(5043))
        

color 19 0.38%
director_name 104 2.06%
num_critic_for_reviews 50 0.99%
duration 15 0.30%
director_facebook_likes 104 2.06%
actor_3_facebook_likes 23 0.46%
actor_2_name 13 0.26%
actor_1_facebook_likes 7 0.14%
gross 884 17.53%
actor_1_name 7 0.14%
actor_3_name 23 0.46%
facenumber_in_poster 13 0.26%
plot_keywords 153 3.03%
num_user_for_reviews 21 0.42%
language 12 0.24%
country 5 0.10%
content_rating 303 6.01%
budget 492 9.76%
title_year 108 2.14%
actor_2_facebook_likes 13 0.26%
aspect_ratio 329 6.52%


In [7]:
# iterate each row; check how many null in each movie
for i in range(0, 5043):
    row = df.iloc[i, :]
    null = row.isnull().sum()
    if null > 10:
        print i, df.loc[i, 'movie_title'], null
        

4 Star Wars: Episode VII - The Force Awakens              14
279 10,000 B.C.              15
2241 Yu-Gi-Oh! Duel Monsters              11
4945 The Brain That Sings  11


# 4. [Cleaning] duplicates removal
Orignial df has 45 duplicates

After pandas.drop_duplicates(), df has 5043-45=4998 rows

In [8]:
df[df.duplicated()].shape


(45, 28)

In [9]:
df = df.drop_duplicates()


In [10]:
df.shape


(4998, 28)

# 5. [Cleaning] correct variables without missing values

### movie_title variable
1. remove weird ending w/ "\wc2\xa0"
2. remove weird space

In [29]:
df.movie_title = df.movie_title.str.replace('\xc2\xa0', '')


In [30]:
df.movie_title = df.movie_title.str.strip(' ')


# 6. [Cleaning] handling variables with low missing value rate 
Clean up variables with low missing value rate 

## color variable (19 null objects)

In [11]:
# extra space in ' Black and White' object
df.color.unique()


array(['Color', nan, ' Black and White'], dtype=object)

In [12]:
df.color = df.color.str.replace(' Black and White', 'Black and White')


In [13]:
# filter "whole dataframe" based on "color is nan"
color_null = df[df.color.isnull()]


In [17]:
# Select title, year and color to decide what to fill for nan
# confirm 19 null are all color movies based on the year
color_null[['movie_title', 'title_year', 'color']].head()


,movie_title,title_year,color
4,Star Wars: Episode VII - The Force Awakens ...,NaN,NaN
279,"10,000 B.C.",NaN,NaN
459,Daredevil,NaN,NaN
1948,Dear John,2010.0,NaN
2015,Red Sky,2014.0,NaN


In [18]:
# fill nan with color
df.color = df.color.fillna(value='Color')


## country variable
fill up 5 missing values

In [19]:
df[df.country.isnull()].shape


(5, 28)

In [20]:
# filter "whole df" based on "country == null"
# use language variable to confirm country variable manually
country_null = df[df.country.isnull()]
country_null[['movie_title', 'language', 'country', 'title_year']]


,movie_title,language,country,title_year
4,Star Wars: Episode VII - The Force Awakens ...,NaN,NaN,NaN
279,"10,000 B.C.",NaN,NaN,NaN
2370,"Gone, Baby, Gone",English,NaN,NaN
3397,Preacher,English,NaN,NaN
4021,Dawn Patrol,English,NaN,2014.0


In [21]:
df.country = df.country.fillna(value='USA')

## language variable
fill up 12 missing values

In [23]:
df[df.language.isnull()].shape


(12, 28)

In [24]:
# filter "whole df" based on "language is null"
language_null = df[df.language.isnull()]
language_null[['movie_title','language', 'country']]


,movie_title,language,country
4,Star Wars: Episode VII - The Force Awakens ...,NaN,USA
279,"10,000 B.C.",NaN,USA
2459,Unforgettable,NaN,USA
3086,September Dawn,NaN,USA
3539,Alpha and Omega 4: The Legend of the Saw Tooth...,NaN,USA
3869,Silent Movie,NaN,USA
4110,Love's Abiding Joy,NaN,USA
4409,Kickboxer: Vengeance,NaN,USA
4630,A Fine Step,NaN,USA
4810,Intolerance: Love's Struggle Throughout the Ages,NaN,USA


In [26]:
# fill na with English
df.language = df.language.fillna(value='English')

# Midpoint1: How does df look like now?

** Budget and Gross columns have around 10% missing values**

In [28]:
for col in df:
    null = df[col].isnull().sum()
    if null > 0:
        print col, null, '{0:.2%}'.format(float(null)/float(len(df)))

director_name 103 2.06%
num_critic_for_reviews 49 0.98%
duration 15 0.30%
director_facebook_likes 103 2.06%
actor_3_facebook_likes 23 0.46%
actor_2_name 13 0.26%
actor_1_facebook_likes 7 0.14%
gross 874 17.49%
actor_1_name 7 0.14%
actor_3_name 23 0.46%
facenumber_in_poster 13 0.26%
plot_keywords 152 3.04%
num_user_for_reviews 21 0.42%
content_rating 301 6.02%
budget 487 9.74%
title_year 107 2.14%
actor_2_facebook_likes 13 0.26%
aspect_ratio 327 6.54%


# 7. [Cleaning] High missing values rate ex: budget and gross

Scraping data from the IMDB websites
* a. scrape information from imdb website                

* b. fill budget or gross by scraped information         


In [31]:
import requests
from bs4 import BeautifulSoup

### budget variable 

fill up 40 missing values

Star Wars: Episode VII - The Force Awakens (index4) and 10,000 B.C. (index279) have wrong links 

Ref: https://stackoverflow.com/questions/9942594/unicodeencodeerror-ascii-codec-cant-encode-character-u-xa0-in-position-20


In [32]:
len(df[df.budget.isnull()])


487

In [33]:
budget_null = df[df.budget.isnull()]

In [34]:
# get movie links whose "budget == null"
budget_null_linklist = list(budget_null['movie_imdb_link'])
len(budget_null_linklist)

487

### DON"T RUN THIS. I already save it to JSON file. If you need the list, turn the Markdown to the Code and run it.

%%time
imdb_budget = {}
for link in budget_null_linklist:
    r = requests.get(link)
    text = r.text
    soup = BeautifulSoup(text)
    budget_location = soup.find(name='h4', attrs={'class':'inline'}, string='Budget:')
    if str(type(budget_location)) == "<class 'bs4.element.Tag'>":
        budget = budget_location.find_previous()
        imdb_budget[link] = float(filter(str.isdigit, budget.text.encode('utf-8')))
    else:
        imdb_budget[link] = np.NaN
        

### save imbd_budget to a json file
with open('imdb_budget.json', 'w') as f:
    json.dump(imdb_budget, f)

In [35]:
with open('imdb_budget.json') as f:
    imdb_budget = json.load(f)
    
len(imdb_budget) # Somwhow it is less than budget_null_linklist


485

In [37]:
# 1. Find a match with imdb_budget dict. by map() in imdb link column
# 2. fillna() in budget column
df.budget = df.movie_imdb_link.map(imdb_budget).fillna(df.budget)


In [38]:
# check how many rows we recover in budget column; only 487 - 447 = 40 rows b/c wrong imdb links or without budget info
len(df[df.budget.isnull()])


447

### During I was fixing the budget variable, there are some movie's link are TV-series. Remove them form the dataset
remove 4998 - 4924 = 74 rows


In [39]:
# remove data whose "content_rating == TV- "
df = df.drop(df[(df.content_rating == 'TV-PG') | (df.content_rating == 'TV-14') | (df.content_rating == 'TV-MA') | (df.content_rating == 'TV-G') | (df.content_rating == 'TV-Y') | (df.content_rating == 'TV-Y7')].index)


In [40]:
# confirm it
df.content_rating.unique()


array(['PG-13', nan, 'PG', 'G', 'R', 'Not Rated', 'Unrated', 'Approved',
       'NC-17', 'X', 'GP', 'Passed', 'M'], dtype=object)

In [41]:
df.shape

(4924, 28)

### title_year variable

fill up 43 missing values (all)

Ref: https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression, https://stackoverflow.com/questions/48793222/pandas-replace-value-in-cells-from-one-column-that-match-condition-in-other-col

 

In [42]:
len(df[df.title_year.isnull()])


43

In [43]:
title_year_null = df[df.title_year.isnull()]
title_year_null_linklist = list(title_year_null['movie_imdb_link'])
len(title_year_null_linklist)


43

### DON"T RUN THIS. I already save it to JSON file. If you need the list, turn the Markdown to the Code and run it.

%%time
imdb_titleyear = {}
for link in title_year_null_linklist:
    r = requests.get(link)
    text = r.text
    soup = BeautifulSoup(text)
    year_location = soup.find(name='h4', attrs={'class':'inline'}, string='Release Date:')
    if str(type(year_location)) == "<class 'bs4.element.Tag'>":
        year = year_location.find_previous()
        pattern = re.compile('\d\d\d\d')
        match = re.search(pattern, year.text.encode('utf-8'))
        imdb_titleyear[link] = int(match.group())
    else:
        imdb_titleyear[link] = np.NaN
        

### save imdb_titleyear into a json file
with open('imdb_titleyear.json', 'w') as f:
    json.dump(imdb_titleyear, f)

In [45]:
with open('imdb_titleyear.json') as f:
    imdb_titleyear = json.load(f)

len(imdb_titleyear)

43

In [46]:
df.title_year = df.movie_imdb_link.map(imdb_titleyear).fillna(df.title_year)


### gross variable 

fill up 802 - 582 = 220 missing values

Ref: https://automatetheboringstuff.com/chapter7/

In [48]:
gross_null = df[df.gross.isnull()]
gross_null_linklist = list(gross_null['movie_imdb_link'])
len(gross_null)

802

### DON"T RUN THIS. I already save it to JSON file. If you need the list, turn the Markdown to the Code and run it.

%%time
imdb_gross = {}
for link in gross_null_linklist:
    r = requests.get(link)
    text = r.text
    soup = BeautifulSoup(text)
    gross_location = soup.find(name='h4', attrs={'class':'inline'}, string='Gross USA:')
    if str(type(gross_location)) == "<class 'bs4.element.Tag'>":
        gross = gross_location.find_previous()
        pattern = re.compile('\$(.*)\W +')
        match = re.search(pattern, gross.text.encode('utf-8'))
        imdb_gross[link] = float(filter(str.isdigit, match.group()))
    else:
        imdb_gross[link] = np.NaN

In [49]:
# Since the dictionary is long, I save into a json file
# Here I can just upload it 
with open('imdb_gross.json') as f:
    imdb_gross = json.load(f)

In [54]:
df.gross = df.movie_imdb_link.map(imdb_gross).fillna(df.gross)

In [55]:
# change 'wrong url' to NaN
df.gross = df.gross.replace('wrong url', np.NaN)

In [56]:
# chekc missing values after scraping
len(df[df.gross.isnull()])

582

# Midpoint2: How does df look like?

In [57]:
for col in df:
    null = df[col].isnull().sum()
    if null > 0:
        print col, null, '{0:.2%}'.format(float(null)/float(len(df)))

director_name 39 0.79%
num_critic_for_reviews 46 0.93%
duration 14 0.28%
director_facebook_likes 39 0.79%
actor_3_facebook_likes 23 0.47%
actor_2_name 13 0.26%
actor_1_facebook_likes 7 0.14%
gross 582 11.82%
actor_1_name 7 0.14%
actor_3_name 23 0.47%
facenumber_in_poster 13 0.26%
plot_keywords 147 2.99%
num_user_for_reviews 20 0.41%
content_rating 301 6.11%
budget 389 7.90%
actor_2_facebook_likes 13 0.26%
aspect_ratio 323 6.56%


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4924 entries, 0 to 5042
Data columns (total 28 columns):
color                        4924 non-null object
director_name                4885 non-null object
num_critic_for_reviews       4878 non-null float64
duration                     4910 non-null float64
director_facebook_likes      4885 non-null float64
actor_3_facebook_likes       4901 non-null float64
actor_2_name                 4911 non-null object
actor_1_facebook_likes       4917 non-null float64
gross                        4342 non-null float64
genres                       4924 non-null object
actor_1_name                 4917 non-null object
movie_title                  4924 non-null object
num_voted_users              4924 non-null int64
cast_total_facebook_likes    4924 non-null int64
actor_3_name                 4901 non-null object
facenumber_in_poster         4911 non-null float64
plot_keywords                4777 non-null object
movie_imdb_link              4924 non-

### Remove all NaN in budget and gross columns

4099 rows left

In [59]:
df = df.dropna(subset=['budget', 'gross'])
df.shape

(4099, 28)

### director_name variable

It has no missing values after the clean up in budget and gross columns

In [60]:
director_name_null = df[df.director_name.isnull()]
director_name_null_linklist = list(director_name_null['movie_imdb_link'])
len(director_name_null)

0

# 8. Export files

### DON"T RUN THIS. I already save it. Check the file in the Github
df.to_csv('wrangle.csv')

# 9. Add new features

## Total gross

**The scraping is not really successful. I only can get the total gross for 2272 movies**

In [69]:
TotalGross_linklist = list(df['movie_imdb_link'])
len(TotalGross_linklist)

4099

In [71]:
imdb_totalgross = {}

In [72]:
%%time
pattern_usd = re.compile('\$(.*)\W +')
pattern_euro = re.compile('\d(.*)\W +')
for link in TotalGross_linklist[0:99]:
    r = requests.get(link)
    text = r.text
    soup = BeautifulSoup(text)
    TotalGross_location = soup.find(name='h4', attrs={'class':'inline'}, string='Cumulative Worldwide Gross:')
    if str(type(TotalGross_location)) == "<class 'bs4.element.Tag'>":
        totalgross = TotalGross_location.find_previous()
        usd_match = re.search(pattern_usd, totalgross.text.encode('utf-8'))
        if str(type(usd_match)) == "<type '_sre.SRE_Match'>":
            imdb_totalgross[link] = float(filter(str.isdigit, usd_match.group()))
        else:
            euro_match = re.search(pattern_euro, totalgross.text.encode('utf-8'))
            euro = float(filter(str.isdigit, euro_match.group()))
            imdb_totalgross[link] = euro*1.15
    else:
        imdb_totalgross[link] = np.NaN
        

CPU times: user 22.7 s, sys: 611 ms, total: 23.3 s
Wall time: 2min 23s


In [17]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in xrange(0, len(l), n):
        yield l[i:i + n]

In [128]:
split = list(chunks(TotalGross_linklist[2000:2999], 400))


In [80]:
def get_total_gross(linklist, destination, original_dic):
    pattern_usd = re.compile('\$(.*)\W +')
    pattern_euro = re.compile('\d(.*)\W +')
    for link in linklist:
        r = requests.get(link)
        text = r.text
        soup = BeautifulSoup(text)
        TotalGross_location = soup.find(name='h4', attrs={'class':'inline'}, string='Cumulative Worldwide Gross:')
        if str(type(TotalGross_location)) == "<class 'bs4.element.Tag'>":
            totalgross = TotalGross_location.find_previous()
            usd_match = re.search(pattern_usd, totalgross.text.encode('utf-8'))
            if str(type(usd_match)) == "<type '_sre.SRE_Match'>":
                destination[link] = float(filter(str.isdigit, usd_match.group()))
                original_dic.update(destination)
            else:
                euro_match = re.search(pattern_euro, totalgross.text.encode('utf-8'))
                euro = float(filter(str.isdigit, euro_match.group()))
                destination[link] = euro*1.15
                original_dic.update(destination)
        else:
            destination[link] = np.NaN  

In [89]:
%%time
temp = {}
for i in split:
    get_total_gross(i, temp, imdb_totalgross)

CPU times: user 3min 40s, sys: 5.18 s, total: 3min 45s
Wall time: 19min 41s


In [144]:
with open('imdb_totalgross.json', 'w') as f:
    json.dump(imdb_totalgross, f)

In [131]:
# add new column
df['total_gross'] = df.movie_imdb_link.map(imdb_totalgross) 

In [137]:
len(df[df.total_gross.isnull()])

1827

In [138]:
df.head()


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,total_gross
0,Color,James Cameron,723,178,0,855,Joel David Moore,1000,760505847,Action|Adventure|Fantasy|Sci-Fi,...,English,USA,PG-13,237000000,2009,936,7.9,1.78,33000,2787965087
1,Color,Gore Verbinski,302,169,563,1000,Orlando Bloom,40000,309404152,Action|Adventure|Fantasy,...,English,USA,PG-13,300000000,2007,5000,7.1,2.35,0,963420425
2,Color,Sam Mendes,602,148,0,161,Rory Kinnear,11000,200074175,Action|Adventure|Thriller,...,English,UK,PG-13,245000000,2015,393,6.8,2.35,85000,880674175
3,Color,Christopher Nolan,813,164,22000,23000,Christian Bale,27000,448130642,Action|Thriller,...,English,USA,PG-13,250000000,2012,23000,8.5,2.35,164000,1084939099
5,Color,Andrew Stanton,462,132,475,530,Samantha Morton,640,73058679,Action|Adventure|Sci-Fi,...,English,USA,PG-13,263700000,2012,632,6.6,2.35,24000,284139100


In [140]:
# remove all rows if total_gross is null
df = df.dropna(subset=['total_gross'])


In [142]:
df.shape


(2272, 29)

In [143]:
# export to csv file
df.to_csv('df_totalgross.csv')

## Date (month+day)

**The scraping is not really successful. I only get 745 targets.**

In [4]:
# get a full list(4099) of imdb links from wrangle.csv file
wrangle = pd.read_csv('wrangle.csv')
df_linklist = list(wrangle.movie_imdb_link)
len(df_linklist)

4099

In [14]:
%%time
imdb_date = {}
for link in df_linklist[0:10]:
    r = requests.get(link)
    text = r.text
    soup = BeautifulSoup(text)
    date_location = soup.find(name='h4', attrs={'class':'inline'}, string='Release Date:')
    if str(type(date_location)) == "<class 'bs4.element.Tag'>":
        date = date_location.find_previous()
        pattern = re.compile('\: +(.*)\d\d\d\d')
        match = re.search(pattern, date.text.encode('utf-8'))
        clean_up = match.group().replace(':', '')
        transform = datetime.datetime.strptime(clean_up, " %d %B %Y")
        imdb_date[link] = [transform.month, transform.day]
    else:
        imdb_date[link] = np.NaN
        

CPU times: user 2.28 s, sys: 73.8 ms, total: 2.35 s
Wall time: 18.1 s


In [15]:
# get a original dictionary with the first 10 rows
imdb_date


{'http://www.imdb.com/title/tt0398286/?ref_=fn_tt_tt_1': [11, 24],
 'http://www.imdb.com/title/tt0401729/?ref_=fn_tt_tt_1': [3, 9],
 'http://www.imdb.com/title/tt0413300/?ref_=fn_tt_tt_1': [5, 4],
 'http://www.imdb.com/title/tt0417741/?ref_=fn_tt_tt_1': [7, 15],
 'http://www.imdb.com/title/tt0449088/?ref_=fn_tt_tt_1': [5, 25],
 'http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1': [12, 18],
 'http://www.imdb.com/title/tt1345836/?ref_=fn_tt_tt_1': [7, 20],
 'http://www.imdb.com/title/tt2379713/?ref_=fn_tt_tt_1': [11, 6],
 'http://www.imdb.com/title/tt2395427/?ref_=fn_tt_tt_1': [5, 1],
 'http://www.imdb.com/title/tt2975590/?ref_=fn_tt_tt_1': [3, 25]}

In [25]:
# split links to different chunks
date_split1 = list(chunks(df_linklist[400:2000], 400))
date_split2 = list(chunks(df_linklist[2001:], 400))

In [21]:
def get_date(linklist, destination, original_dic):
    for link in linklist:
        r = requests.get(link)
        text = r.text
        soup = BeautifulSoup(text)
        date_location = soup.find(name='h4', attrs={'class':'inline'}, string='Release Date:')
        if str(type(date_location)) == "<class 'bs4.element.Tag'>":
            date = date_location.find_previous()
            pattern = re.compile('\: +(.*)\d\d\d\d')
            match = re.search(pattern, date.text.encode('utf-8'))
            clean_up = match.group().replace(':', '')
            
            transform = datetime.datetime.strptime(clean_up, " %d %B %Y")
            destination[link] = [transform.month, transform.day]
            original_dic.update(destination)
        else:
            destination[link] = np.NaN
            

In [28]:
len(imdb_date)

745

In [26]:
%%time
temp={}
for i in date_split1:
    get_date(i, temp, imdb_date)

KeyboardInterrupt: 

In [301]:
with open('imdb_date.json', 'w') as f:
    json.dump(imdb_date, f)

In [280]:
df['date'] = df.movie_imdb_link.map(imdb_date)

In [292]:
df['day'] = df.date.str[0]
df['month'] = df.date.str[1]

In [297]:
# df = df.drop(['column_name'], axis=1) --> remove column
len(df[df.date.isnull()])

657

In [298]:
# remove all rows if date, or day, month is null
df = df.dropna(subset=['total_gross', 'day', 'month'])

In [320]:
# 1615 rows left
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1615 entries, 0 to 5012
Data columns (total 32 columns):
color                        1615 non-null object
director_name                1615 non-null object
num_critic_for_reviews       1615 non-null float64
duration                     1615 non-null float64
director_facebook_likes      1615 non-null float64
actor_3_facebook_likes       1614 non-null float64
actor_2_name                 1615 non-null object
actor_1_facebook_likes       1615 non-null float64
gross                        1615 non-null float64
genres                       1615 non-null object
actor_1_name                 1615 non-null object
movie_title                  1615 non-null object
num_voted_users              1615 non-null int64
cast_total_facebook_likes    1615 non-null int64
actor_3_name                 1614 non-null object
facenumber_in_poster         1612 non-null float64
plot_keywords                1614 non-null object
movie_imdb_link              1615 non-

In [300]:
df.to_csv('df_totalgross_date.csv')

## Month

In [66]:
# get a full list(4099) of imdb links from wrangle.csv file
wrangle = pd.read_csv('wrangle.csv')
df_linklist = list(wrangle.movie_imdb_link)
len(df_linklist)

4099

In [ ]:
# 1st scraping codes.

In [71]:
%%time
imdb_month = {}
for link in df_linklist[0:10]:
    r = requests.get(link)
    text = r.text
    soup = BeautifulSoup(text)
    month_location = soup.find(name='h4', attrs={'class':'inline'}, string='Release Date:')
    if str(type(month_location)) == "<class 'bs4.element.Tag'>":
        month = month_location.find_previous()
        pattern = re.compile('\d +(.*) +\d\d\d\d')
        match = re.findall(pattern, month.text.encode('utf-8'))
        imdb_month[link] = match
    else:
        imdb_month[link] = np.NaN

CPU times: user 2.39 s, sys: 342 ms, total: 2.73 s
Wall time: 23.6 s


In [72]:
imdb_month

{'http://www.imdb.com/title/tt0398286/?ref_=fn_tt_tt_1': ['November'],
 'http://www.imdb.com/title/tt0401729/?ref_=fn_tt_tt_1': ['March'],
 'http://www.imdb.com/title/tt0413300/?ref_=fn_tt_tt_1': ['May'],
 'http://www.imdb.com/title/tt0417741/?ref_=fn_tt_tt_1': ['July'],
 'http://www.imdb.com/title/tt0449088/?ref_=fn_tt_tt_1': ['May'],
 'http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1': ['December'],
 'http://www.imdb.com/title/tt1345836/?ref_=fn_tt_tt_1': ['July'],
 'http://www.imdb.com/title/tt2379713/?ref_=fn_tt_tt_1': ['November'],
 'http://www.imdb.com/title/tt2395427/?ref_=fn_tt_tt_1': ['May'],
 'http://www.imdb.com/title/tt2975590/?ref_=fn_tt_tt_1': ['March']}

In [73]:
# 2nd scraping function based on 1st one

def get_month(linklist, destination, original_dic):
    for link in linklist:
        r = requests.get(link)
        text = r.text
        soup = BeautifulSoup(text)
        month_location = soup.find(name='h4', attrs={'class':'inline'}, string='Release Date:')
        if str(type(month_location)) == "<class 'bs4.element.Tag'>":
            month = month_location.find_previous()
            pattern = re.compile('\d +(.*) +\d\d\d\d')
            match = re.findall(pattern, month.text.encode('utf-8'))
            if str(type(match)) == "<type '_sre.SRE_Match'>":
                destination[link] = match
                original_dic.update(destination)
            elif:
                pattern_backup = re.compile('\W +(.*) +\d\d\d\d')
                match = re.findall(pattern_backup, month.text.encode('utf-8'))
                destination[link] = match
                original_dic.update(destination)
            else:
                destination[link] = np.NaN
        else:
            destination[link] = np.NaN
            

SyntaxError: invalid syntax (<ipython-input-73-ae0942a69758>, line 16)

### Split links to different chunks for scraping function

In [39]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in xrange(0, len(l), n):
        yield l[i:i + n]

In [40]:
date_split1 = list(chunks(df_linklist[11:2000], 300))
date_split2 = list(chunks(df_linklist[2001:], 300))

In [ ]:
### DON"T RUN THIS. I already save it to JSON file. If you need the list, turn the Markdown to the Code and run it.

%%time
temp={}
for i in date_split1:
    get_month(i, temp, imdb_month)

### save imbd_month to a json file

with open('imdb_month.json', 'w') as f:
    json.dump(imdb_month, f)

In [62]:
with open('imdb_month.json') as f:
    imdb_month = json.load(f)

len(imdb_month)


3657

# ---------------------------------------------------------------------

In [89]:
%%time
imdb_month = {}
for link in df_linklist[0:10]:
    r = requests.get(link)
    text = r.text
    soup = BeautifulSoup(text)
    month_location = soup.find(name='h4', attrs={'class':'inline'}, string='Release Date:')
    if str(type(month_location)) == "<class 'bs4.element.Tag'>":
        month = month_location.nextSibling
        imdb_month[link] = month
    else:
        imdb_month[link] = np.NaN

CPU times: user 2.57 s, sys: 236 ms, total: 2.8 s
Wall time: 18.9 s


In [90]:
imdb_month

{'http://www.imdb.com/title/tt0398286/?ref_=fn_tt_tt_1': u' 24 November 2010 (USA)\n    ',
 'http://www.imdb.com/title/tt0401729/?ref_=fn_tt_tt_1': u' 9 March 2012 (USA)\n    ',
 'http://www.imdb.com/title/tt0413300/?ref_=fn_tt_tt_1': u' 4 May 2007 (USA)\n    ',
 'http://www.imdb.com/title/tt0417741/?ref_=fn_tt_tt_1': u' 15 July 2009 (USA)\n    ',
 'http://www.imdb.com/title/tt0449088/?ref_=fn_tt_tt_1': u' 25 May 2007 (USA)\n    ',
 'http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1': u' 18 December 2009 (USA)\n    ',
 'http://www.imdb.com/title/tt1345836/?ref_=fn_tt_tt_1': u' 20 July 2012 (USA)\n    ',
 'http://www.imdb.com/title/tt2379713/?ref_=fn_tt_tt_1': u' 6 November 2015 (USA)\n    ',
 'http://www.imdb.com/title/tt2395427/?ref_=fn_tt_tt_1': u' 1 May 2015 (USA)\n    ',
 'http://www.imdb.com/title/tt2975590/?ref_=fn_tt_tt_1': u' 25 March 2016 (USA)\n    '}

In [95]:
def get_released_date(linklist, destination):
    for link in linklist:
        r = requests.get(link)
        text = r.text
        soup = BeautifulSoup(text)
        month_location = soup.find(name='h4', attrs={'class':'inline'}, string='Release Date:')
        if str(type(month_location)) == "<class 'bs4.element.Tag'>":
            month = month_location.nextSibling
            destination[link] = month
        else:
            destination[link] = np.NaN

In [92]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in xrange(0, len(l), n):
        yield l[i:i + n]

In [94]:
date_split1 = list(chunks(df_linklist[0:2000], 300))
date_split2 = list(chunks(df_linklist[2001:], 300))

In [100]:
%%time
imdb_month={}
for i in date_split1:
    get_released_date(i, imdb_month)

CPU times: user 8min 17s, sys: 5min 15s, total: 13min 33s
Wall time: 1h 15min 39s


In [103]:
len(imdb_month)

1989

In [ ]:
%%time
imdb_month_2={}
for i in date_split2:
    get_released_date(i, imdb_month_2)

In [112]:
imdb_month_2

{'http://www.imdb.com/title/tt0095889/?ref_=fn_tt_tt_1': u' 10 June 1988 (USA)\n    ',
 'http://www.imdb.com/title/tt0165831/?ref_=fn_tt_tt_1': u' 12 May 2000 (USA)\n    ',
 'http://www.imdb.com/title/tt0114825/?ref_=fn_tt_tt_1': u' 27 October 1995 (USA)\n    ',
 'http://www.imdb.com/title/tt0096320/?ref_=fn_tt_tt_1': u' 9 December 1988 (USA)\n    ',
 'http://www.imdb.com/title/tt0120772/?ref_=fn_tt_tt_1': u' 17 April 1998 (USA)\n    ',
 'http://www.imdb.com/title/tt0926129/?ref_=fn_tt_tt_1': u' 11 April 2008 (USA)\n    ',
 'http://www.imdb.com/title/tt0375679/?ref_=fn_tt_tt_1': u' 6 May 2005 (USA)\n    ',
 'http://www.imdb.com/title/tt1596346/?ref_=fn_tt_tt_1': u' 8 April 2011 (USA)\n    ',
 'http://www.imdb.com/title/tt0104940/?ref_=fn_tt_tt_1': u' 11 December 1992 (USA)\n    ',
 'http://www.imdb.com/title/tt0218619/?ref_=fn_tt_tt_1': u' 7 September 2001 (USA)\n    ',
 'http://www.imdb.com/title/tt0404203/?ref_=fn_tt_tt_1': u' 9 February 2007 (USA)\n    ',
 'http://www.imdb.com/title

In [85]:
r = requests.get('https://www.imdb.com/title/tt0398286/?ref_=fn_tt_tt_1')
text = r.text
soup = BeautifulSoup(text)
month_location = soup.find(name='h4', attrs={'class':'inline'}, string='Release Date:')

In [88]:
month_location.nextSibling

u' 24 November 2010 (USA)\n    '